# PM2.5 Dataset from 2021 to 2023 analysis

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, avg, max, min, count, sum as spark_sum,
    year, month, dayofweek, when, lit, round as spark_round,
    greatest, coalesce, dense_rank, row_number
)
from pyspark.sql.window import Window
from pyspark.sql.types import (
    StructType, StructField, 
    StringType, IntegerType, DoubleType, DateType
)
import time

# Start Spark
# Most basic Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PM25").getOrCreate()


## Load data

In [46]:
# Load PM2.5 data
pm25_df = spark.read \
    .option("header", "true") \
    .csv("data/epa_raw/daily_88101_*.csv")

pm25_df.show(5)

+----------+-----------+--------+--------------+---+---------+----------+-----+--------------------+---------------+------------------+----------+--------------------+----------+-----------------+-------------------+---------------+-------------+------------+---+-----------+--------------------+-----------------+--------------------+----------+-----------+---------+--------------------+-------------------+
|State Code|County Code|Site Num|Parameter Code|POC| Latitude| Longitude|Datum|      Parameter Name|Sample Duration|Pollutant Standard|Date Local|    Units of Measure|Event Type|Observation Count|Observation Percent|Arithmetic Mean|1st Max Value|1st Max Hour|AQI|Method Code|         Method Name|  Local Site Name|             Address|State Name|County Name|City Name|           CBSA Name|Date of Last Change|
+----------+-----------+--------+--------------+---+---------+----------+-----+--------------------+---------------+------------------+----------+--------------------+----------+--

25/11/11 23:36:10 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: data/epa_raw/daily_88101_*.csv.
java.io.FileNotFoundException: File data/epa_raw/daily_88101_*.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.

## Transformation type, filter, Join and group by operation 

In [ ]:
# filtering PM2.5 data
pm25_filtered = pm25_df

# column type casting and transformation
pm25_typed = (pm25_filtered
    .withColumn("date_local", F.to_date(col("Date Local")))              
    .withColumn("arith_mean_d", col("Arithmetic Mean").cast("double"))    
    .withColumn("aqi_i",        col("AQI").cast("int"))                  
    .withColumn("obs_pct_d",    col("Observation Percent").cast("double"))
)

pm25_filtered = (pm25_typed
    .filter(col("date_local") >= F.lit("2021-01-01").cast("date"))   
    .filter(col("arith_mean_d").isNotNull() &
            (col("arith_mean_d") >= 0) &
            (col("arith_mean_d") < 500))                            
    .filter(col("aqi_i").isNotNull())
    .filter(col("obs_pct_d") >= F.lit(75.0))                         
    .filter(col("CBSA Name").isNotNull() & col("City Name").isNotNull())
)

# join to find stations in the sma city
city_stations = pm25_filtered.groupBy("State Name", "City Name").agg(
    count("*").alias("measurement_count"),
    avg("Arithmetic Mean").alias("city_avg")
)

result_df = pm25_filtered.join(city_stations, ["State Name", "City Name"], "left")

result_df.show(5)


+----------+---------+----------+-----------+--------+--------------+---+---------+----------+-----+--------------------+---------------+------------------+----------+--------------------+----------+-----------------+-------------------+---------------+-------------+------------+---+-----------+--------------------+-----------------+--------------------+-----------+--------------------+-------------------+----------+------------+-----+---------+-----------------+-----------------+
|State Name|City Name|State Code|County Code|Site Num|Parameter Code|POC| Latitude| Longitude|Datum|      Parameter Name|Sample Duration|Pollutant Standard|Date Local|    Units of Measure|Event Type|Observation Count|Observation Percent|Arithmetic Mean|1st Max Value|1st Max Hour|AQI|Method Code|         Method Name|  Local Site Name|             Address|County Name|           CBSA Name|Date of Last Change|date_local|arith_mean_d|aqi_i|obs_pct_d|measurement_count|         city_avg|
+----------+---------+------

## Column transformation with Pyspark

In [50]:
# Transformations to enrich data
enriched_df = result_df \
    .withColumn("Year", year(col("Date Local"))) \
    .withColumn("Month", month(col("Date Local"))) \
    .withColumn("Season",
        when(col("Month").isin([12, 1, 2]), "Winter")
        .when(col("Month").isin([3, 4, 5]), "Spring")
        .when(col("Month").isin([6, 7, 8]), "Summer")
        .otherwise("Fall")
    ) \
    .withColumn("AQI_Category",
        when(col("AQI") <= 50, "Good")
        .when(col("AQI") <= 100, "Moderate")
        .when(col("AQI") <= 150, "Unhealthy for Sensitive")
        .otherwise("Unhealthy")
    ) \
    .withColumn("PM25_Rounded", spark_round(col("Arithmetic Mean"), 2))

## SQL queries

In [51]:
# df table
enriched_df.createOrReplaceTempView("pm25_data")

# 1. Top polluted cities
# Fixed SQL query
query1 = spark.sql("""
    SELECT 
        `State Name`,
        `City Name`,
        ROUND(AVG(`Arithmetic Mean`), 2) as Avg_PM25,
        COUNT(*) as Days
    FROM pm25_data
    GROUP BY `State Name`, `City Name`
    HAVING COUNT(*) >= 100
    ORDER BY Avg_PM25 DESC
    LIMIT 10
""")
query1.show()

+----------+-----------+--------+----+
|State Name|  City Name|Avg_PM25|Days|
+----------+-----------+--------+----+
|California|Bakersfield|   15.56|2471|
|California|    Visalia|   15.55|1050|
|    Oregon|   Oakridge|   14.24|1133|
|California|    Hanford|   14.13|1075|
|California|    Ontario|   13.99|1938|
|California|     Fresno|   13.66|3473|
|California|   Corcoran|   13.13|1070|
|California|  Otay Mesa|   12.98| 788|
|California|    Modesto|   12.96|1064|
|California|  Mira Loma|   12.92|2291|
+----------+-----------+--------+----+



In [60]:
# 2. Monthly trends
query2 = spark.sql("""
    SELECT 
        Year,
        Month,
        ROUND(AVG(`Arithmetic Mean`), 2) as Monthly_Avg,
        COUNT(*) as Measurements
    FROM pm25_data
    GROUP BY Year, Month
    ORDER BY Year, Month
""")
query2.show()

+----+-----+-----------+------------+
|Year|Month|Monthly_Avg|Measurements|
+----+-----+-----------+------------+
|2021|    1|       8.72|       32534|
|2021|    2|       8.52|       28847|
|2021|    3|       7.38|       31524|
|2021|    4|       7.65|       31928|
|2021|    5|        6.9|       33238|
|2021|    6|       7.49|       32120|
|2021|    7|      11.53|       33642|
|2021|    8|      11.79|       34049|
|2021|    9|       8.46|       33130|
|2021|   10|        6.8|       34749|
|2021|   11|       8.09|       33589|
|2021|   12|       8.56|       34837|
|2022|    1|       8.89|       35093|
|2022|    2|       8.02|       32188|
|2022|    3|       6.98|       35398|
|2022|    4|       6.57|       34553|
|2022|    5|       7.03|       35985|
|2022|    6|       8.01|       35230|
|2022|    7|       7.71|       36320|
|2022|    8|       6.79|       36299|
+----+-----+-----------+------------+
only showing top 20 rows


## Optimize

In [61]:
# Only keep columns we actually use
enriched_df = pm25_filtered.select(
    "State Code",
    "County Code", 
    "Site Num",
    "Date Local",
    "Arithmetic Mean",
    "AQI",
    "State Name",
    "City Name"
)

enriched_df.write \
    .mode("overwrite") \
    .parquet("output/pm25_enriched")

query1.write.mode("overwrite").parquet("output/top_cities")
query2.write.mode("overwrite").parquet("output/monthly_trends")

25/11/11 23:54:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


## Performance Analysis

In [63]:
enriched_df.explain(mode="formatted")

== Physical Plan ==
* Project (3)
+- * Filter (2)
   +- Scan csv  (1)


(1) Scan csv 
Output [10]: [State Code#3112, County Code#3113, Site Num#3114, Date Local#3123, Observation Percent#3127, Arithmetic Mean#3128, AQI#3131, State Name#3136, City Name#3138, CBSA Name#3139]
Batched: false
Location: InMemoryFileIndex [file:/Users/yuqianwang/Documents/IDS706/ids706_pyspark_data_processing/data/epa_raw/daily_88101_2021.csv, ... 2 entries]
PushedFilters: [IsNotNull(Date Local), IsNotNull(Arithmetic Mean), IsNotNull(AQI), IsNotNull(Observation Percent), IsNotNull(CBSA Name), IsNotNull(City Name)]
ReadSchema: struct<State Code:string,County Code:string,Site Num:string,Date Local:string,Observation Percent:string,Arithmetic Mean:string,AQI:string,State Name:string,City Name:string,CBSA Name:string>

(2) Filter [codegen id : 1]
Input [10]: [State Code#3112, County Code#3113, Site Num#3114, Date Local#3123, Observation Percent#3127, Arithmetic Mean#3128, AQI#3131, State Name#3136, City Name#3138